In [2]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Conv1D

2023-09-02 22:49:16.651934: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-02 22:49:17.096625: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
BATCH_SIZE = 1
IN_DIM = 2
K = 3
OUT_DIM = 4

x = np.random.random((BATCH_SIZE, K, IN_DIM))
x.shape, x

((1, 3, 2),
 array([[[0.89528223, 0.77096866],
         [0.52790267, 0.66903583],
         [0.85850122, 0.1493018 ]]]))

In [5]:
c1d = Conv1D(filters=OUT_DIM, kernel_size=K, use_bias=False)

In [6]:
y = c1d(x)
assert y.shape == (BATCH_SIZE, 1, OUT_DIM)
y

2023-09-02 22:49:32.807325: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-02 22:49:32.832917: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


<tf.Tensor: shape=(1, 1, 4), dtype=float32, numpy=
array([[[-0.2512602 ,  0.40120566,  0.0269018 ,  0.60123014]]],
      dtype=float32)>

In [7]:
kernel = c1d.weights[0].numpy()
kernel.shape, kernel

((3, 2, 4),
 array([[[-0.4412105 ,  0.2847181 , -0.43272266,  0.19800508],
         [ 0.5563215 , -0.38290536,  0.12355131, -0.17165372]],
 
        [[-0.12209055, -0.45529744,  0.50919354,  0.02319765],
         [-0.43101716,  0.5168401 ,  0.22561276,  0.26710588]],
 
        [[-0.00859588,  0.37832463, -0.06686699,  0.3998586 ],
         [ 0.50259745,  0.07559645, -0.28992128,  0.1478265 ]]],
       dtype=float32))

In [8]:
batchless_x = x[0]

batchless_x.shape, batchless_x

((3, 2),
 array([[0.89528223, 0.77096866],
        [0.52790267, 0.66903583],
        [0.85850122, 0.1493018 ]]))

In [10]:
result = np.einsum('ki,kij->j', batchless_x, kernel)
result.shape, result

((4,), array([-0.2512602 ,  0.40120567,  0.02690176,  0.60123015]))

In [18]:
batchless_x = batchless_x.reshape((K, 1, IN_DIM))
np.matmul(batchless_x, kernel).sum(axis=-1)

array([[-0.25363039],
       [ 0.36331111],
       [ 0.66839666]])

array([[[ 0.03389852, -0.04030498, -0.29215472,  0.04493079],
        [-0.44160622, -0.00915219,  0.62981229,  0.2266987 ],
        [ 0.37979114,  0.39698981, -0.28338505,  0.47195589]],

       [[ 0.13928281, -0.10587396, -0.14577519, -0.01031508],
        [-0.35281785,  0.10543181,  0.41974765,  0.1909495 ],
        [ 0.33171791,  0.25029532, -0.22926699,  0.30998764]],

       [[-0.29571996,  0.18726237, -0.3530465 ,  0.14435939],
        [-0.16916652, -0.31370825,  0.47082767,  0.0597946 ],
        [ 0.06765913,  0.33607884, -0.10069117,  0.36534985]]])

In [ ]:
batchless_x = x[0]
result = np.zeros((out_dim,))
for k in range(K):
    sub_result = np.dot(batchless_x[k], kernel[k])
    print(sub_result)
    result += sub_result
result

In [ ]:
np.dot(batchless_x[0], kernel[0])

In [ ]:
for k in [0,1,2]:
    print("k", k)
    a = batchless_x[k]
    b = kernel[k]
    print(a.shape, a)
    print(b.shape, b)
    print(np.dot(a, b))